In [ ]:
import pandas as pd
import re

In [ ]:
f = open('WhatsApp Chat with Subject.txt', 'r', encoding='utf-8')

In [ ]:
data = f.read()

In [ ]:
# print(data)

pattern = r'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s'

In [ ]:
messages = re.split(pattern, data)[1:]
# messages
len(messages)

In [ ]:
dates = re.findall(pattern, data)
# dates
len(dates)

In [ ]:
df = pd.DataFrame({'user_message': messages, 'message_date': dates})

# convert messsage_date type
df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %H:%M - ')

df.rename(columns = {'message_date': 'date'}, inplace = True)
df.head()

In [ ]:
df.shape

In [ ]:
# separate users and messages
users = []
messages = []
for message in df['user_message']:
    entry = re.split(r'([\w\W]+?):\s', message)
    # print(entry)
    if entry[1:]: # username
        users.append(entry[1])
        messages.append(entry[2])
    else:
        users.append('whatsapp_notification')
        messages.append(entry[0])

# print (len(messages))
df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df.head()

In [ ]:
# separate year, moth, day, hour, minute columns
df['year'] = df['date'].dt.year

In [ ]:
df['month'] = df['date'].dt.month_name()

In [ ]:
df['day'] = df['date'].dt.day

In [ ]:
df['hour'] = df['date'].dt.hour

In [ ]:
df['minute'] = df['date'].dt.minute

In [ ]:
df.head()

In [ ]:
# select specific user
df[df['user'] == 'Ayushi'].shape

In [ ]:
# count words
words = []
for message in df['message']:
    words.extend(message.split())

In [ ]:
len(words)

In [ ]:
# For url count
from urlextract import URLExtract

extractor = URLExtract()
# Adds the links in a list
urls = extractor.find_urls("www.gmail.com youtube.com google.com")

urls

In [ ]:
# Fetching shared url count
links = []

for message in df['message']:
    # Returns a list containing the urls shared
    links.extend(extractor.find_urls(message))

len(links)

In [ ]:
# 3. Fetching number of media messages
num_media_messages = df[df['message'] == '<Media omitted>\n'].shape[0]

num_media_messages

In [ ]:
# counts number of times each user occurs (desc order) - used to calculate most active users
# top 5 users
x = df['user'].value_counts().head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
name = x.index
count = x.values

In [ ]:
# Bar graph for most active users
plt.bar(name, count)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# Calculating percentage of contribution of each user out of total messages
round((df['user'].value_counts() / df.shape[0]) * 100, 2).reset_index().rename(columns = {'count': 'percent'})